### Importer
Med "importer" gjør vi nåværende python "bevisst" på kodepakker som allerede er innstallerte, av feks. Google, inn i "colab".

Pandas og Plotly er eksempler på pakker som ikke ligger i standardbiblioteket, men som Google har innstallert inn i Colab for oss. Denne kodesnutten gjør pakkene tilgjengelig for oss i notebooket vårt:

In [ ]:
import pandas as pd
import plotly.express as px
import requests

### Hent kommuneinndelt norgeskart 
Vi henter innholdet i en "geojson" fil fra github, med kommuneinndelingen fra 2020.

In [ ]:
kommune_json_path = "https://raw.githubusercontent.com/robhop/fylker-og-kommuner-2020/master/Kommuner-medium.json"
geojson = requests.get(kommune_json_path).json()
#geojson  # Se på objektet om du vil...

### Hent data fra github
Google colab lar oss hente ting fra github, her har vi lagt en csv-fil rett på githuben vår.
Metoden "read_csv" forstår hva en webadresse er, og forsøker å hente data derfra.\
Fordi CSVen er europeisk, så må vi spesifisere seperatoren til semikolon (i USA bruker de komma).\
Python liker alt som "UTF8" encodet, og siden csven er encodet som "latin1", så må vi spesifisere dette.\
Vi ønsker å bruke første rad som "kolonnenavnene", og setter da header=1.

In [ ]:
# Hent dataene 
kommune_url = "https://raw.githubusercontent.com/statisticsnorway/kurs-forskningskvelden/main/data/valg_2019_18_19aar.csv"
kommune_data = pd.read_csv(kommune_url,
                    encoding="latin1",
                    sep=";",
                    header=1)
kommune_data

### Modifiser dataene så den lettere lar seg grafe

In [ ]:
kommune_data = kommune_data.rename(columns={"Valgdeltakelse 2019 Begge kjønn":"valgdeltakelse"})
kommune_data["valgdeltakelse"] = (kommune_data["valgdeltakelse"]
                                  .replace(":", float('nan')) # SSB bruker tegn som : og . for å si at celler er tomme
                                  .replace(".", float('nan'))
                                  .astype(float) # konverter kolonnen til ett ekte desimaltall
                                 )
kommune_data["kommnr"] = kommune_data["region"].str[:4] # Lager en ny kolonne med kun kommnr fra region-kolonnen
kommune_data

### Filtrer geojson ned til kun de brukte kommunenummerene i dataene


In [ ]:
geojson_features_in_data = [feature for feature in geojson["features"] 
                            if str(feature["properties"]["kommunenummer"]).zfill(4) in kommune_data["kommnr"]]
geojson_in_data = geojson.copy()
geojson_in_data["features"] = geojson_features_in_data

### Graf ett kart med dataene og geojson-innholdet.

In [ ]:
fig = px.choropleth(kommune_data,
                    geojson=geojson, 
                    locations=f"kommnr",
                    labels={"region" : "Kommune"},
                    featureidkey = "properties.kommunenummer",
                    color="valgdeltakelse",
                    projection="mercator",
                    scope="europe",
                    height=1500,
                    width=1500,
                    hover_data=["region", "valgdeltakelse"]
                      )
# Change settings on figure
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)')
     

# Oppgaver:
- Kan du forklare hvorfor noen kommuner er hvite?
- Utforsk parametrene til plotly choropleth, er det noe mer du kan justere for at kartet skal bli mer som du vil?
- Hva skjer om du filtrerer ned dataene før du lager kartet? Får du til å begrense det ned til kommunene i Innlandet? Se i forrige notebook for kodesnutt.
- Finn annen data du er interessert i på kommunenivå, og prøv å få det inn i notebooket og lage ett fint kart basert på det.